In [2]:
# Read centroids.npy
import numpy as np

centroids = np.load('centroids.npy')
print(centroids)

[10041562808529065984  2965059885268957184  7638391071187194880 ...
  7754781312546421760  3433926456038300160  7965685133453433856]


In [3]:
len(centroids)

20000

In [4]:
def load_binary_file(file):
    """
    Load a numpy array from a binary file
    """
    with open(file, 'rb') as f:
        packed_data = f.read()
    data = np.frombuffer(packed_data, dtype=np.uint64)
    # return np.unpackbits(data, axis=-1, bitorder='little')
    return data


In [5]:
def hamming_distance(a, b):
    """Calculate the Hamming distance using bit_count() (Python 3.10+)."""
    return (a ^ b).bit_count()

In [6]:
cluster_assignments = np.load('cluster_assignments.npy')

In [8]:
# Count the number of elements in each cluster
cluster_counts = np.bincount(cluster_assignments)
print(cluster_counts)

[  0 257   0 ...  45   0   5]


In [28]:
# Write the cluster counts to a file in the required format
with open('cluster_counts.txt', 'w') as f:
    for count in cluster_counts:
        f.write(f"{count}\n")


In [2]:
# For cluster 9, calculate average distance to other elements in the same cluster (randomly selected points instead of pairwise)
cluster_9_indices = np.where(cluster_assignments == 9)[0]
print(cluster_9_indices)

NameError: name 'cluster_assignments' is not defined

In [19]:
cluster_0_indices = np.where(cluster_assignments == 0)[0]
print(cluster_0_indices)
print(len(cluster_0_indices))

[]
0


In [7]:
cluster_5236_indices = np.where(cluster_assignments == 5236)[0]
print(cluster_5236_indices)

[   89985   307852   308721 ... 60326563 60326570 60326825]


In [8]:
len(cluster_5236_indices)

23292

In [9]:
cluster_5252_indices = np.where(cluster_assignments == 5252)[0]
print(cluster_5252_indices)

[     176      187      196 ... 60799536 60801094 60801399]


In [10]:
len(cluster_5252_indices)

2938354

In [13]:
all_binary = load_binary_file('output_set.bin')

In [12]:
def calculate_avg_hamming(cluster_indices):
    total_distance = 0
    num_samples = 1000
    random_indices = np.random.choice(cluster_indices, num_samples, replace=False)
    count = 0
    pair_points = []
    
    for i in range(num_samples):
        for j in range(i + 1, num_samples):
            tmp_distance = hamming_distance(all_binary[random_indices[i]], all_binary[random_indices[j]])
            if tmp_distance < 5:
                print(f"Hamming distance: {tmp_distance}")
                print(f"Points: {random_indices[i]}, {random_indices[j]}")
                print(f"Values: {all_binary[random_indices[i]]}, {all_binary[random_indices[j]]}")
                pair_points.append((random_indices[i], random_indices[j]))
            # elif tmp_distance < 10:
            #     print("Hamming distance less than 10")
            total_distance += tmp_distance
            count += 1
    
    return total_distance / count, pair_points

In [118]:
def calculate_avg_hamming_points(points):
    # Points is the actual binary data
    total_distance = 0
    num_samples = 2000
    random_indices = np.random.choice(points, num_samples, replace=False)
    count = 0
    
    for i in range(num_samples):
        for j in range(i + 1, num_samples):
            tmp_distance = hamming_distance(random_indices[i], random_indices[j])
            if tmp_distance < 5:
                print(f"Hamming distance: {tmp_distance}")
                print(f"Points: {random_indices[i]}, {random_indices[j]}")
            total_distance += tmp_distance
            count += 1
            
    return total_distance / count, pair_points

In [32]:
def calculate_avg_hamming_to_anchor(anchor, points):
    # Points is the actual binary data
    total_distance = 0
    num_samples = 10000
    random_indices = np.random.choice(points, num_samples, replace=False)
    count = 0
    
    for i in range(num_samples):
        tmp_distance = hamming_distance(anchor, all_binary[random_indices[i]])
        if tmp_distance < 10:
            print(f"Hamming distance: {tmp_distance}")
            print(f"Points: {random_indices[i]}")
            print(f"Values: {all_binary[random_indices[i]]}")
        total_distance += tmp_distance
        count += 1
            
    return total_distance / count

In [21]:
centroids[5236]

2975458558343465472

In [33]:
all_binary[cluster_5236_indices]

array([ 9516804762356247151,  9519054363146676751,  9519054363750656513,
       ..., 17715605822399933141, 17715605822467074645,
       17715607334161295967], dtype=uint64)

In [35]:
calculate_avg_hamming_to_anchor(centroids[5236], cluster_5236_indices)

22.2335

In [37]:
len(cluster_5236_indices)

23292

In [8]:
calculate_avg_hamming_points(centroids)

NameError: name 'calculate_avg_hamming_points' is not defined

In [36]:
total_distance, pair_points = calculate_avg_hamming(cluster_5236_indices)

Hamming distance: 4
Points: 50992827, 50992491
Values: 16526587214151101141, 16526587213614246493
Hamming distance: 4
Points: 50992827, 47158903
Values: 16526587214151101141, 16454597789297224405
Hamming distance: 3
Points: 50992827, 47120891
Values: 16526587214151101141, 16454529620113189461
Hamming distance: 4
Points: 50992827, 51009878
Values: 16526587214151101141, 16526657582928833117
Hamming distance: 2
Points: 50992827, 50951706
Values: 16526587214151101141, 16526024264231234261
Hamming distance: 3
Points: 50992827, 50978866
Values: 16526587214151101141, 16526094632941857365
Hamming distance: 4
Points: 50992827, 50978884
Values: 16526587214151101141, 16526094632941906645
Hamming distance: 4
Points: 50992827, 51009738
Values: 16526587214151101141, 16526657582895282781
Hamming distance: 4
Points: 50992827, 21229676
Values: 16526587214151101141, 11914883603537685213
Hamming distance: 4
Points: 50992827, 47121087
Values: 16526587214151101141, 16454529620180281877
Hamming distance: 4


In [86]:
# Use faiss for performing a search for the top 1 nearest neighbors
# Within a cluster, 
# find the nearest neighbor to anchor, call it B
# then find the nearest neighbor to B, call it C
# ...

import faiss
from tqdm import tqdm

# Perform a search for the top 1 nearest neighbors
def search_nearest_neighbors(centroid, cluster_indices):
    num_points = len(cluster_indices)
    
    # Create an index
    index = faiss.IndexBinaryFlat(64)
    # all_binary[cluster_indices] is uint64, convert to uint8
    b_uint8 = all_binary[cluster_indices].view(np.uint8).reshape(-1, 8)
    index.add(b_uint8)
    
    # centroid is uint64 object, convert to uint8
    centroid_uint8 = np.array([centroid]).view(np.uint8).reshape(1, -1)
    
    D, I = index.search(centroid_uint8, k=1)
    print(f"Nearest neighbors: {I} with distance {D}")
    
    current_point_index = I[0][0]
    print(f"Current points: {current_point_index}")
    path = [current_point_index]
    cost = [D[0][0]]
    
    for i in tqdm(range(num_points)):
        # Remove the current point from the index
        index.remove_ids(np.array([current_point_index], dtype=np.int64))
        
        # Perform a search
        D, I = index.search(np.array([all_binary[current_point_index]]).view(np.uint8).reshape(1, -1), k=1)
        current_point_index = I[0][0]
        path.append(current_point_index)
        cost.append(D[0][0])
        
    return path, cost
        
path, cost = search_nearest_neighbors(centroids[5236], cluster_5236_indices)

Nearest neighbors: [[527]] with distance [[17]]
Current points: 527


  0%|          | 0/23292 [00:00<?, ?it/s]

100%|██████████| 23292/23292 [01:15<00:00, 306.99it/s] 


In [87]:
path

[527,
 107,
 199,
 379,
 2297,
 7,
 40,
 65,
 429,
 89,
 618,
 103,
 619,
 2283,
 40,
 2383,
 141,
 1575,
 215,
 196,
 2759,
 89,
 2276,
 1968,
 96,
 198,
 73,
 636,
 877,
 70,
 87,
 2293,
 0,
 2271,
 1670,
 85,
 98,
 327,
 45,
 184,
 96,
 268,
 568,
 48,
 9713,
 33,
 83,
 405,
 185,
 1930,
 385,
 773,
 2371,
 289,
 1683,
 2881,
 1722,
 371,
 391,
 316,
 81,
 10176,
 634,
 2718,
 372,
 2768,
 294,
 1694,
 3042,
 9138,
 2485,
 34,
 181,
 5,
 692,
 1562,
 51,
 2710,
 670,
 392,
 381,
 598,
 363,
 1556,
 128,
 174,
 2311,
 608,
 2226,
 9116,
 19275,
 2454,
 1525,
 179,
 9665,
 9921,
 1679,
 498,
 365,
 13,
 636,
 2413,
 56,
 572,
 2695,
 1682,
 222,
 37,
 1726,
 591,
 2703,
 84,
 10116,
 56,
 2689,
 207,
 87,
 9737,
 146,
 10676,
 116,
 85,
 189,
 370,
 58,
 169,
 10585,
 2563,
 115,
 56,
 262,
 2723,
 19273,
 74,
 15763,
 1511,
 503,
 267,
 347,
 284,
 278,
 10,
 38,
 3366,
 316,
 295,
 356,
 2290,
 2673,
 2206,
 1852,
 108,
 1857,
 57,
 2788,
 297,
 8,
 2177,
 9643,
 9872,
 100,
 340,
 

In [88]:
cost

[17,
 8,
 11,
 10,
 8,
 8,
 13,
 8,
 11,
 10,
 11,
 8,
 11,
 10,
 7,
 8,
 11,
 14,
 9,
 8,
 10,
 6,
 11,
 7,
 9,
 13,
 11,
 11,
 13,
 14,
 10,
 10,
 7,
 8,
 8,
 9,
 11,
 12,
 6,
 11,
 14,
 13,
 11,
 9,
 13,
 10,
 10,
 12,
 11,
 15,
 14,
 13,
 10,
 7,
 10,
 10,
 9,
 8,
 7,
 14,
 7,
 13,
 9,
 12,
 9,
 7,
 6,
 9,
 11,
 8,
 12,
 8,
 10,
 14,
 10,
 9,
 9,
 10,
 9,
 11,
 11,
 10,
 9,
 7,
 10,
 10,
 14,
 9,
 7,
 9,
 10,
 10,
 8,
 7,
 9,
 9,
 9,
 10,
 8,
 10,
 11,
 13,
 9,
 11,
 10,
 11,
 10,
 7,
 10,
 9,
 11,
 7,
 13,
 10,
 11,
 11,
 12,
 10,
 13,
 10,
 10,
 12,
 11,
 13,
 10,
 13,
 13,
 10,
 11,
 12,
 12,
 9,
 12,
 10,
 12,
 8,
 8,
 10,
 8,
 10,
 12,
 11,
 9,
 10,
 9,
 8,
 11,
 8,
 8,
 8,
 11,
 10,
 12,
 10,
 13,
 8,
 12,
 8,
 8,
 11,
 12,
 15,
 9,
 11,
 12,
 11,
 11,
 12,
 9,
 9,
 9,
 9,
 8,
 10,
 8,
 7,
 9,
 10,
 10,
 10,
 13,
 10,
 8,
 10,
 11,
 10,
 11,
 9,
 13,
 8,
 7,
 9,
 7,
 10,
 6,
 11,
 7,
 11,
 7,
 11,
 11,
 7,
 11,
 9,
 9,
 8,
 10,
 8,
 13,
 10,
 11,
 9,
 14,
 9,
 7,
 9,
 11,
 8,

In [17]:
len(pair_points)

7497

In [18]:
pair_points = np.array(pair_points)

# Save the pair points to a file
np.save('pair_points.npy', pair_points)


In [121]:
calculate_avg_hamming(cluster_5252_indices)

Hamming distance: 4
Points: 36942201, 37965298
Values: 15290366848673902749, 15308398839402983577
Hamming distance: 4
Points: 36942201, 37936135
Values: 15290366848673902749, 15308381247216941205
Hamming distance: 3
Points: 36942201, 37938010
Values: 15290366848673902749, 15308381248257126549
Hamming distance: 4
Points: 51334806, 51335104
Values: 16533359813795361467, 16533359814332232367
Hamming distance: 4
Points: 36712885, 29512477
Values: 15284121090222045731, 14128947631182690851
Hamming distance: 4
Points: 55228204, 38212486
Values: 17615364999712946871, 15309521853060299287
Hamming distance: 4
Points: 42751126, 45347004
Values: 15380931439612504755, 15416960236631466677
Hamming distance: 4
Points: 42751126, 42798889
Values: 15380931439612504755, 15380949031796468387
Hamming distance: 4
Points: 42751126, 42800623
Values: 15380931439612504755, 15380949031966321329
Hamming distance: 3
Points: 42596207, 43049259
Values: 15380368472644891297, 15381494372417516197
Hamming distance: 3


15.55489089089089

: 

In [87]:
# Calculate the average hamming distance to other elements in the same cluster
total_distance = 0
num_samples = 1000
random_indices = np.random.choice(cluster_9_indices, num_samples, replace=False)

for i in range(num_samples - 1):
    a = all_binary[random_indices[i]]
    b = all_binary[random_indices[i + 1]]
    tmp_distance = hamming_distance(a, b)
    if tmp_distance == 0:
        print("Hamming distance is 0")
    elif tmp_distance < 5:
        print("Hamming distance less than 5")
        print(f"Points: {random_indices[i]}, {random_indices[i + 1]}")
    elif tmp_distance < 10:
        print("Hamming distance less than 10")
    total_distance += tmp_distance

average_distance = total_distance / (num_samples - 1)
print(f"Average Hamming distance: {average_distance}")

Hamming distance less than 10
Hamming distance less than 10
Hamming distance less than 10
Hamming distance less than 10
Hamming distance less than 10
Hamming distance less than 10
Hamming distance less than 10
Hamming distance less than 10
Hamming distance less than 10
Hamming distance less than 10
Hamming distance less than 10
Hamming distance less than 10
Hamming distance less than 10
Hamming distance less than 10
Hamming distance less than 10
Hamming distance less than 10
Hamming distance less than 10
Hamming distance less than 10
Hamming distance less than 10
Hamming distance less than 10
Hamming distance less than 10
Hamming distance less than 10
Hamming distance less than 10
Hamming distance less than 10
Hamming distance less than 10
Hamming distance less than 10
Hamming distance less than 10
Hamming distance less than 10
Hamming distance less than 10
Hamming distance less than 10
Hamming distance less than 10
Hamming distance less than 10
Hamming distance less than 10
Hamming di

In [ ]:
# Create an FAISS index for binary data
binary_index = faiss.IndexBinaryFlat(64)

# Prepare binary data for the index
binary_data_uint8 = binary_data[subset_indices].view(np.uint8).reshape(-1, 8)
binary_index.add(binary_data_uint8)

# Prepare query data (e.g., centroid) for searching
query_data_uint8 = np.array([query_vector]).view(np.uint8).reshape(1, -1)

# Perform a search for the nearest neighbor
distances, neighbors = binary_index.search(query_data_uint8, k=1)